In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [ ]:
print("🚀 Starting Master Pipeline...")



# --- 1. LOAD DATA ---
df = pd.read_csv("fps_benchmark.csv")
print(f"✅ Data Loaded. Shape: {df.shape}")




# --- 2. CLEAN TEXT & RESOLUTION (The Smart Way) ---

# Standardize text columns
text_cols = ['CpuName', 'GpuName', 'GameName', 'GameResolution']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower()

# Smart Resolution Mapper (Handles 1080p, FHD, 1920x1080, etc.)
def get_resolution_score(res_str):
    s = str(res_str)
    if '4k' in s or '2160' in s or 'uhd' in s: return 4
    if '1440' in s or '2k' in s or 'qhd' in s: return 3
    if '1080' in s or 'fhd' in s: return 2
    if '720' in s or 'hd' in s: return 1
    return None # If unknown

df['GameResolution_Enc'] = df['GameResolution'].apply(get_resolution_score)

# Drop rows where resolution is unknown
df.dropna(subset=['GameResolution_Enc'], inplace=True)
print(f"✅ Resolutions Mapped. Rows remaining: {len(df)}")

# Clean Target (Remove weird 0 FPS or 9999 FPS)
df = df[(df['FPS'] > 5) & (df['FPS'] < 500)]





# --- 3. SAVE LOOKUP TABLES (The "Brains") ---
# We grab the specs before we filter X, so the app knows what "RTX 3060" means.
# Dynamically find numeric columns for CPU and GPU
gpu_cols = [c for c in df.columns if 'gpu' in c.lower() and df[c].dtype in [np.float64, np.int64]]
cpu_cols = [c for c in df.columns if 'cpu' in c.lower() and df[c].dtype in [np.float64, np.int64]]

# Save GPU Specs
gpu_specs = df.groupby('GpuName')[gpu_cols].mean().reset_index()
gpu_specs.to_csv('gpu_specs_lookup.csv', index=False)

# Save CPU Specs
cpu_specs = df.groupby('CpuName')[cpu_cols].mean().reset_index()
cpu_specs.to_csv('cpu_specs_lookup.csv', index=False)
print("✅ Lookup Tables Saved (gpu_specs_lookup.csv, cpu_specs_lookup.csv)")




# --- 4. PREPARE MODEL DATA ---
# Encode Game Name
le_game = LabelEncoder()
df['GameName_Enc'] = le_game.fit_transform(df['GameName'])
with open('game_encoder.pkl', 'wb') as f:
    pickle.dump(le_game, f)

# SELECT FEATURES (The Fix for "Cannot convert string to float")
# We only keep Numbers. This automatically drops 'GpuName', 'GpuBus', etc.
X = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32'])

# Drop Target from X
X = X.drop(columns=['FPS'], errors='ignore')

# Handle NaNs (Simple median fill)
X = X.fillna(X.median())

y = df['FPS']

print(f"✅ Features Selected: {list(X.columns)}")






# --- 5. TRAIN ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("⏳ Training Random Forest... (Hold tight)")
model = RandomForestRegressor(n_estimators=80, max_depth=20, random_state=42)
model.fit(X_train, y_train)

# Save Model
with open('fps_model.pkl', 'wb') as f:
    pickle.dump(model, f)




# --- 6. RESULTS ---
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))

print("-" * 30)
print(f"🎉 SUCCESS! Model Trained.")
print(f"🎯 FINAL RMSE: {rmse:.2f} FPS")
print("-" * 30)

🚀 Starting Master Pipeline...
✅ Data Loaded. Shape: (24624, 44)
✅ Resolutions Mapped. Rows remaining: 24624
✅ Lookup Tables Saved (gpu_specs_lookup.csv, cpu_specs_lookup.csv)
✅ Features Selected: ['CpuNumberOfCores', 'CpuNumberOfThreads', 'CpuBaseClock', 'CpuCacheL1', 'CpuCacheL2', 'CpuCacheL3', 'CpuDieSize', 'CpuFrequency', 'CpuMultiplier', 'CpuProcessSize', 'CpuTDP', 'CpuNumberOfTransistors', 'CpuTurboClock', 'GpuBandwidth', 'GpuBaseClock', 'GpuBoostClock', 'GpuNumberOfComputeUnits', 'GpuDieSize', 'GpuNumberOfExecutionUnits', 'GpuFP32Performance', 'GpuMemoryBus', 'GpuMemorySize', 'GpuPixelRate', 'GpuProcessSize', 'GpuNumberOfROPs', 'GpuNumberOfShadingUnits', 'GpuNumberOfTMUs', 'GpuTextureRate', 'GpuNumberOfTransistors', 'GameResolution_Enc', 'GameName_Enc']
⏳ Training Random Forest... (Hold tight)
------------------------------
🎉 SUCCESS! Model Trained.
🎯 FINAL RMSE: 26.22 FPS
------------------------------


In [ ]:
import pickle

# Save the exact list of columns the model was trained on
feature_names = X.columns.tolist()

with open('model_columns.pkl', 'wb') as f:
    pickle.dump(feature_names, f)

print("✅ Feature Order Saved! Now you are ready for the app.")

✅ Feature Order Saved! Now you are ready for the app.


In [ ]:
import joblib
import pickle

# 1. Load your existing big model
with open('fps_model.pkl', 'rb') as f:
    model = pickle.load(f)

# 2. Save it with HIGH compression
# This usually shrinks file size by 5x-10x
joblib.dump(model, 'fps_model.pkl', compress=9)

print("✅ Model Compressed! Download the new 'fps_model.pkl' and try uploading.")

✅ Model Compressed! Download the new 'fps_model.pkl' and try uploading.
